# Parameter Space

Let's look at the parameter space

In [1]:
import dynamite as dyn

fname = 'NGC6278_config.yaml'
c = dyn.config_reader.Configuration(fname, silent=True)

parset = c.parspace.get_parset()

model = dyn.model.LegacySchwarzschildModel(
    system=c.system,
    settings=c.settings,
    parspace=c.parspace,
    executor=c.executor,
    parset=parset)

No previous models have been found:
Making an empty table in AllModels.table


For now, let's extract a single parameter set to be able to create a model. For every parameter, in the configuration file we have specified a ``value``. Let's look at these values. To access them, we can loop over ``parspace``, which is a list of all the parameters,

In [2]:
print('Parameter / value :')
for par in c.parspace:
    print(f'   {par.name} = {par.value}')

Parameter / value :
   mass = 6.0
   a = 0.001
   dc = 10.0
   f = 100.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 3.0


This is almost the ``parset`` that we need to make the model. One complication is that some parameters are specified in logarithmic units, i.e. 
```
parameters -> XXX -> logarithmic : True
```
This can be useful for parameters (e.g. masses) for which it makes sense to take logarithmically spaced steps through parameter space. For other parameters (e.g. length scales) linearly spaced steps may be more appropriate. For other types of parameters (e.g. angles) a different spacing may be preferable.

To handle these possibilities, we introduce the concept of ``raw`` parameter values, distinct from the values themselves. All values associated with parameter in the configuration file are given in ``raw`` units. When we step through parameter space, we take linear steps in ``raw`` values. The conversion from raw values to the parameter values is handles by the method
```
Parameter.get_par_value_from_raw_value
```
So to convert the above list from raw values, we can do the following,

In [3]:
print('Parameter / value :')
for par in c.parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    print(f'   {par.name} = {par_value}')

Parameter / value :
   mass = 1000000.0
   a = 0.001
   dc = 10.0
   f = 100.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 3.0


Lastly, we must pass a parameter set to the ``Model`` in the form of a row of an ``Astropy Table``, which we can create as follows,

In [4]:
from astropy.table import Table

t = Table()
for par in c.parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    t[par.name] = [par_value]
# extract 0th - i.e. the only - row from the table
parset = t[0]

print(parset)

   mass     a    dc    f    q    p     u     ml
--------- ----- ---- ----- ---- ---- ------ ---
1000000.0 0.001 10.0 100.0 0.54 0.99 0.9999 3.0
